# Interactive Energy Profile Visualization Tool  
  
This Jupyter Notebook provides an interactive tool for visualizing and exploring energy profiles. It allows you to analyze time-series data for individual households across the following categories:  
- **Consumption**: Energy usage data  
- **PV (Photovoltaic)**: Solar power generation profiles  
- **EV (Electric Vehicle)**: Charging profiles for electric vehicles  
- **HP (Heat Pump)**: Energy usage for heat pumps  
  
You can examine data for specific households aggregated over an entire year, or zoom in to explore the details for a single day.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
%matplotlib inline


def update_customer_options(change):
    """
    Updates the customer dropdown when the profile type changes.
    """
    profile_type = change.new
    new_customer_list = profiles[profile_type].columns.tolist()
    customer_dropdown.options = new_customer_list


def on_view_toggle_change(change):
    """Handles logic when switching between 'Yearly' and 'Daily' view."""
    if change.new == 'Daily':
        day_dropdown.disabled = False
    else:
        day_dropdown.disabled = True
    # Trigger a replot when the view changes
    on_value_change(None)


def on_value_change(change):
    """Main function to redraw the plot based on all widget values."""
    with plot_output:
        clear_output(wait=True)  # Clear the previous plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Get current values from all widgets
        profile_type = profile_type_dropdown.value
        customer = customer_dropdown.value
        view_mode = view_toggle.value
        selected_day = day_dropdown.value
        
        if customer not in profiles[profile_type].columns:
            ax.text(0.5, 0.5, 'No data to display.', ha='center', va='center', transform=ax.transAxes)
            plt.show()
            return

        series = profiles[profile_type][customer]
        
        if view_mode == 'Yearly':
            series_to_plot = series
            ax.set_title(f"Yearly {profile_type} Profile for {customer}", fontsize=16)
            ax.set_xlabel("Date", fontsize=12)
        else: # Daily view
            # Filter the series for the selected day
            series_to_plot = series.loc[selected_day]
            ax.set_title(f"Daily {profile_type} Profile for {customer} on {selected_day}", fontsize=16)
            ax.set_xlabel("Time of Day", fontsize=12)
            
        ax.plot(series_to_plot.index, series_to_plot.values, label=f"{profile_type} - {customer}")
        ax.set_ylabel("Power (kW)", fontsize=12)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)
        ax.legend()
        fig.autofmt_xdate()
        
        plt.tight_layout()
        plt.show()

# Data Loading
data_path = "../blob-account-edh/challenge-data/data_parquet/"
df_consumption = pd.read_parquet(data_path + "base_consumption.parquet").set_index('timestamp')
df_pv = pd.read_parquet(data_path + "pv_profiles.parquet").set_index('timestamp')
df_ev = pd.read_parquet(data_path + "ev_profiles.parquet").set_index('timestamp')
df_hp = pd.read_parquet(data_path + "hp_profiles.parquet").set_index('timestamp')
print("Successfully loaded and indexed all profiles data.")

# Data Preparation and Timestamp Mapping
profiles = {"Consumption": df_consumption, "PV": df_pv, "EV": df_ev, "HP": df_hp}
num_periods = len(df_consumption.index)
datetime_index = pd.date_range(start='2050-01-01', periods=num_periods, freq='15min')
for df in profiles.values():
    df.index = datetime_index
print(f"\nCreated new datetime index from {datetime_index.min()} to {datetime_index.max()}")


# Create the Interactive Widgets and Output Area
profile_type_dropdown = widgets.Dropdown(options=list(profiles.keys()), value='Consumption', description='Profile Type:', style={'description_width': 'initial'})
initial_customers = profiles[profile_type_dropdown.value].columns.tolist()
customer_dropdown = widgets.Dropdown(options=initial_customers, description='Customer/Asset:', style={'description_width': 'initial'})
view_toggle = widgets.RadioButtons(
    options=['Yearly', 'Daily'],
    value='Yearly',
    description='View:',
    disabled=False
)
unique_days = sorted(list(set(datetime_index.date)))
day_options = [day.strftime('%Y-%m-%d') for day in unique_days]
day_dropdown = widgets.Dropdown(
    options=day_options,
    description='Select Day:',
    style={'description_width': 'initial'},
    disabled=True  # Disabled by default, will be enabled in 'Daily' mode
)
plot_output = widgets.Output()


# Link dropdowns
profile_type_dropdown.observe(update_customer_options, names='value')
profile_type_dropdown.observe(on_value_change, names='value')
customer_dropdown.observe(on_value_change, names='value')
view_toggle.observe(on_view_toggle_change, names='value')
day_dropdown.observe(on_value_change, names='value')

# Arrange controls into two rows for better layout
controls_row1 = widgets.HBox([profile_type_dropdown, customer_dropdown])
controls_row2 = widgets.HBox([view_toggle, day_dropdown])
ui = widgets.VBox([controls_row1, controls_row2, plot_output])

print("\nInteractive visualization is ready. Please use the dropdowns below.")
display(ui)

# Trigger the first plot draw manually
on_value_change(None)

Successfully loaded and indexed all profiles data.

Created new datetime index from 2050-01-01 00:00:00 to 2050-12-30 23:45:00

Interactive visualization is ready. Please use the dropdowns below.
